In [ ]:
import requests
from hashlib import md5

class Chaojiying_Client(object):

    def __init__(self, username, password, soft_id):
        self.username = username
        password = password.encode('utf8')

        self.password = md5(password).hexdigest()
        self.soft_id = soft_id
        self.base_params = {
            'user': self.username,
            'pass2': self.password,
            'softid': self.soft_id,
        }
        self.headers = {
            'Connection': 'Keep-Alive',
            'User-Agent': 'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0)',
        }

    def PostPic(self, im, codetype):
        """
        im: 图片字节
        codetype: 题目类型 参考 http://www.chaojiying.com/price.html
        """
        params = {
            'codetype': codetype,
        }
        params.update(self.base_params)
        files = {'userfile': ('ccc.jpg', im)}
        r = requests.post('http://upload.chaojiying.net/Upload/Processing.php', data=params, files=files,
                          headers=self.headers)
        return r.json()

    def ReportError(self, im_id):
        """
        im_id:报错题目的图片ID
        """
        params = {
            'id': im_id,
        }
        params.update(self.base_params)
        r = requests.post('http://upload.chaojiying.net/Upload/ReportError.php', data=params, headers=self.headers)
        return r.json()

In [ ]:
# 以上为超级鹰文档代码

from selenium import webdriver
from time import sleep
from PIL import Image
from selenium.webdriver import ActionChains
import random

# 使用selenium打开登录界面
bro = webdriver.Chrome(executable_path='./chromedriver')


def login():
    bro.get('https://passport.bilibili.com/login')

    # 定位用户名，密码输入框
    username = bro.find_element_by_id('login-username')
    password = bro.find_element_by_id('login-passwd')
    username.send_keys('xxxxx')
    password.send_keys('xxxxx')

    # 定位登录按钮并点击
    bro.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
    sleep(random.random()*3)

def save_img():
    # save_screenshot就是将当前页面进行截图且保
    bro.save_screenshot('page.png')
    # 定位验证码图片的位置
    code_img_ele = bro.find_element_by_xpath('/html/body/div[2]/div[2]/div[6]/div/div')
    location = code_img_ele.location  # 验证码左上角的坐标x,y
    size = code_img_ele.size  # 验证码图片对应的长和宽

    # 左上角和右下角的坐标
    rangle = (
        int(location['x'] * 1.25), int(location['y'] * 1.25), int((location['x'] + size['width']) * 1.25),
        int((location['y'] + size['height']) * 1.25)
    )

    i = Image.open('./page.png')
    code_img_name = './code.png'
    # crop根据rangle元组内的坐标进行裁剪
    frame = i.crop(rangle)
    frame.save(code_img_name)
    return code_img_ele

def narrow_img():
    # 缩小图片
    code = Image.open('./code.png')
    small_img = code.resize((169, 216))
    small_img.save('./small_img.png')
    print(code.size, small_img.size)

def submit_img():
    # 将验证码提交给超级鹰进行识别
    chaojiying = Chaojiying_Client('1837751407', '123456', '908533')  # 用户中心>>软件ID 生成一个替换 96001
    im = open('small_img.png', 'rb').read()  # 本地图片文件路径 来替换 a.jpg 有时WIN系统须要//
    print(chaojiying.PostPic(im, 9004)['pic_str'])
    result = chaojiying.PostPic(im, 9004)['pic_str']
    return result

def parse_data(result):
    all_list = []  # 存储即将被点击的点的坐标  [[x1,y1],[x2,y2]]
    if '|' in result:
        list_1 = result.split('|')
        count_1 = len(list_1)
        for i in range(count_1):
            xy_list = []
            x = int(list_1[i].split(',')[0])
            y = int(list_1[i].split(',')[1])
            xy_list.append(x)
            xy_list.append(y)
            all_list.append(xy_list)
    else:
        print(result.split(',')[0])
        print(result.split(',')[1])

        x = int(result.split(',')[0])
        y = int(result.split(',')[1])
        xy_list = []
        xy_list.append(x)
        xy_list.append(y)
        all_list.append(xy_list)
    return all_list

def click_codeImg(all_list, code_img_ele):
    # 遍历列表，使用动作链对每一个列表元素对应的x,y指定的位置进行点击操作
    for l in all_list:
        x = l[0] * 1.6
        y = l[1] * 1.6
        # move_to_element_with_offset移动到距某个元素（左上角坐标）多少距离的位置
        ActionChains(bro).move_to_element_with_offset(code_img_ele, x, y).click().perform()
        sleep(random.random())
        print('点击已完成')

    sleep(random.random()*2)
    # 完成动作链点击操作后，定位确认按钮并点击
    bro.find_element_by_xpath('/html/body/div[2]/div[2]/div[6]/div/div/div[3]/a').click()

def main():
    # 进入登录界面，输入账号密码
    login()
    # 保存页面截图，并根据坐标裁剪获取验证码图片
    code_img_ele = save_img()
    # 缩小图片
    narrow_img()
    # 将图片提交给超级鹰,获取返回值
    result = submit_img()
    # 解析返回值,将数据格式化
    all_list = parse_data(result)
    # 在页面验证码上完成点击操作
    click_codeImg(all_list, code_img_ele)

main()